<a href="https://colab.research.google.com/github/atloveu1226/CMIT2024/blob/main/CMIT_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import linprog
import time

We study the case when we set the restriction on horizontal and vertical elements. The below are some preparition:


In [23]:
b   = [3, 2, 3, 3, 2, 2, 1, 0, 1, 1, 3, 2, 1, 0, 0]

Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

beq = np.array(b[:8])

c = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



lb = np.zeros(15)

ub = np.ones(15)


In [24]:
result = linprog(
    c,
    A_eq = Aeq,
    b_eq = beq,
    bounds = list(zip(lb, ub)),
    method = 'highs',
   )
print(result.x)

#x = [1, 1, 0, 0, 0, 1, 0, 1, 0, 0,1, 1, 1, 0, 0]

#print(np.dot(Aeq,x) == b[:8])

[ 1.  0.  1.  1.  0.  1.  1. -0.  0.  0.  1.  1.  1. -0.  0.]


In this case, we shall define **7** different states, according the diagonal sum, namely, we define the 'STATE' function.

In [25]:
def STATE(x :np.array):

  Dia_ob = b[8:15]

  state = np.zeros(len(Dia_ob))

  count = 0

  i = 0

  #Translate diagonal restriction into matrix

  M = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

  # Get the temporary variable

  tem = np.dot(M, x)

  for i in range(len(Dia_ob)):

    if tem[i] == Dia_ob[i]:

      count = count + 1

  state[count - 1] = 1

  return state

Then, it is necessary to define the step function (i.e. from one state to next state)

In [26]:
#Example to test the ability of this function

#x_0 = np.array([1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0])

#x_1 = np.array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0])

#print(STATE(x_0))

#print(STATE(x_1))

In [28]:
def OBJECT(c : np.array, Aeq, beq):

    new_c = np.zeros(len(c))

    result = linprog(
    c,
    A_eq = Aeq,
    b_eq = beq,
    bounds = list(zip(lb, ub)),
    method = 'highs-ds',
   )

    solution = result.x

    mean = np.mean(solution)

    for i in range(len(solution)):

      new_c[i - 1] = (solution[i - 1]- mean) ** 2

    return [result, new_c]

#solution: OBJECT(c)[0]
#new_c : OBJECT(c)[1]



In [17]:
#print(OBJECT(c, Aeq, beq)[0])
print(Aeq.shape, beq.shape)

(8, 15) (8,)


In [29]:
obs_size = 7 #7
n_actions = 7  #it has 7 different kind of actions
HIDDEN_SIZE = 120

net= nn.Sequential(
            nn.Linear(obs_size, HIDDEN_SIZE),
            nn.ReLU(),
            #nn.Sigmoid(),
            nn.Linear(HIDDEN_SIZE, n_actions)
        )

objective = nn.CrossEntropyLoss() #quite standard for classification tasks
optimizer = optim.Adam(params=net.parameters(), lr=0.001)

In [30]:
sm = nn.Softmax(dim=1) #Softmax converts the 7-dimensional output vector to a probability distribution

def select_action(c, Aeq, beq):

        x = OBJECT(c, Aeq, beq)[0].x

        state = STATE(x)

        state_t = torch.FloatTensor([state])

        act_probs_t = sm(net(state_t))

        #print(act_probs_t)

        act_probs = act_probs_t.data.numpy()[0]

        action = np.random.choice(len(act_probs), p=act_probs) #chooses randomly one of the 4 actions according to the probabilities returned by the net

        return action


In [31]:
#Explaination for above method

#print(type(select_action(c, Aeq, beq)))

Then, it is necessary to define the STEP function:

In [32]:
def next_state(c, next_action, Aeq, beq):

  reward = 0

  flag = False

  current_state = 0

  nextstage_ = 0

  x = OBJECT(c, Aeq, beq)[0].x

  state = STATE(x)

  result = np.zeros(len(state))

  for i in range(len(state)):

    if state[i] == 1:

      current_state = i

      break

  next_state_int = current_state + next_action

  if OBJECT(c, Aeq, beq)[0].success == True:

    if next_state_int <= len(state) - 1:

      result[next_state_int] = 1

    else:

      result[(next_state_int + 2) % 7] = 1

  else:

    result = state

  for j in range(len(state)):

    if result[j] == 1:

      nextstage_ = j

      break

  if nextstage_ == 6:

    reward = 1

    flag = True

  elif nextstage_ < current_state:

    reward = 0

    flag = True

  else:

    reward = 0

    flag = False

  #print(current_state)


  return result, reward, flag, nextstage_ + 1


In [33]:
for i in range(10):

  next_action = select_action(c, Aeq, beq)

  print(next_action)

  print(next_state(c, next_action, Aeq, beq))

print(type(next_state(c, next_action, Aeq, beq)))

0
(array([0., 0., 0., 0., 0., 0., 1.]), 1, True, 7)
2
(array([0., 0., 0., 1., 0., 0., 0.]), 0, True, 4)
6
(array([1., 0., 0., 0., 0., 0., 0.]), 0, True, 1)
6
(array([1., 0., 0., 0., 0., 0., 0.]), 0, True, 1)
6
(array([1., 0., 0., 0., 0., 0., 0.]), 0, True, 1)
4
(array([0., 0., 0., 0., 0., 1., 0.]), 0, True, 6)
5
(array([0., 0., 0., 0., 0., 0., 1.]), 1, True, 7)
5
(array([0., 0., 0., 0., 0., 0., 1.]), 1, True, 7)
4
(array([0., 0., 0., 0., 0., 1., 0.]), 0, True, 6)
0
(array([0., 0., 0., 0., 0., 0., 1.]), 1, True, 7)
<class 'tuple'>


<ipython-input-30-1f004a66f62a>:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  state_t = torch.FloatTensor([state])


Then we need to generate new restrction matrix from new state.

In [34]:
def Matrix_Generator(Aeq, beq, n):
    result1 = []

    result2 = []

    random_condition = []

    Store1 = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

    Store2   = b[8:]

    index = [9, 10, 11, 12, 13, 14, 15]

    random_index = np.sort(np.random.choice(index, n, replace=False))

    #print(random_index)

    for i in range(n):

        result1.append(Store1[random_index[i] - 9])

        result2.append(Store2[random_index[i] - 9])

    result1 = np.array(result1)

    result2 = np.array(result2)

    new_Aeq = np.concatenate((Aeq, result1), axis = 0)

    if len(new_Aeq) >= 15:

      new_Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

    new_beq = np.concatenate((beq, result2), axis = 0)

    if len(new_beq) >= 15:

      new_beq = b


    return [new_Aeq, new_beq]

In [27]:
new = Matrix_Generator(Aeq, beq, 7)

print(new[0], new[1])

[[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]] [2, 2, 3, 3, 2, 2, 0, 0, 1, 2, 1, 2, 1, 0, 0]


In [35]:
BATCH_SIZE = 200 #200

GAMMA = 0.9

PERCENTILE = 30 #30
REWARD_GOAL = 0.8

from collections import namedtuple  #more readable tuples
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [36]:
start_time = time.time()

iter_no = 0
reward_mean = 0
full_batch = []
batch = []
episode_steps = []
episode_reward = 0.0
x = OBJECT(c, Aeq, beq)[0].x
state = STATE(x)

while reward_mean < REWARD_GOAL:

        action = select_action(c, Aeq, beq)
        #next_state, reward, episode_is_done, _ , _= env.step(action)
        nextstate_, reward, episode_is_done, current_state = next_state(c, action, Aeq, beq)

        episode_steps.append(EpisodeStep(observation=nextstate_, action=action))
        episode_reward += reward

        #print(episode_steps, episode_is_done)
        #print(episode_reward)

        if episode_is_done: # Episode finished
            batch.append(Episode(reward=episode_reward, steps=episode_steps))


            #print(len(batch))

            Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

            beq = np.array(b[:8])

            c = [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]

            episode_steps = []
            episode_reward = 0.0

            if len(batch) == BATCH_SIZE: # New set of batches ready --> select "elite"

                #print("Batch full")
                #print(batch)
                #print("\n")
                #input("Press Enter to continue...")


                reward_mean = float(np.mean(list(map(lambda s: s.reward, batch)))) #compute mean reward (lambda is inline function)
                elite_candidates= batch
                #elite_candidates= batch + full_batch
                returnG = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), elite_candidates))
                reward_bound = np.percentile(returnG, PERCENTILE) #lowest score that is greater than PERCENTILE% of scores in the data set
                                                                  #Keep the highest 100-PERCENTILE %
                #print("Batch finished", returnG, reward_bound)
                #input("Press Enter to continue...")


                train_obs = []
                train_act = []
                elite_batch = []

                for example, discounted_reward in zip(elite_candidates, returnG):
                        if discounted_reward > reward_bound:
                        #if discounted_reward >= reward_bound:
                              train_obs.extend(map(lambda step: step.observation, example.steps))
                              train_act.extend(map(lambda step: step.action, example.steps))
                              elite_batch.append(example)
                full_batch=elite_batch
                state=train_obs
                acts=train_act

                #print(state)
                #print(acts)
                #input("Press Enter to continue...")

                #Do the training
                if len(full_batch) != 0 : # just in case empty during an iteration
                  state_t = torch.FloatTensor(state) #batch of states: [[1.0,0,0,0,0,0,0,0,0,0],[1,...]]
                  acts_t = torch.LongTensor(acts) # batch of actions: [0,2,3,1,..]

                  #print(state_t)
                  #print(acts_t)
                  #input("Press Enter to continue...")
                  optimizer.zero_grad() #it is good practice to do this, initializing the gradient computations
                  action_scores_t = net(state_t)

                  #print(action_scores_t)
                  #input("Press Enter to continue...")

                  loss_t = objective(action_scores_t, acts_t)
                  loss_t.backward() #computes the gradients
                  optimizer.step() #updates the weights according to the gradients
                  print("%d: loss=%.3f, reward_mean=%.3f" % (iter_no, loss_t.item(), reward_mean))
                  iter_no += 1
                batch = [] #empty the batch

        #state = next_state

        c = OBJECT(c, Aeq, beq)[1]

        new = Matrix_Generator(Aeq, beq, current_state)

        Aeq = new[0]

        beq = new[1]

        new_x = OBJECT(c, Aeq, beq)[0].x

        state = STATE(new_x)

Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

beq = np.array(b[:8])

c = [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]


print("--- %s seconds ---" % (time.time() - start_time))

0: loss=1.889, reward_mean=0.260
1: loss=1.865, reward_mean=0.280
2: loss=1.836, reward_mean=0.330
3: loss=1.810, reward_mean=0.325
4: loss=1.783, reward_mean=0.375
5: loss=1.766, reward_mean=0.370
6: loss=1.741, reward_mean=0.335
7: loss=1.721, reward_mean=0.390
8: loss=1.692, reward_mean=0.365
9: loss=1.673, reward_mean=0.400
10: loss=1.643, reward_mean=0.380
11: loss=1.623, reward_mean=0.355
12: loss=1.600, reward_mean=0.450
13: loss=1.580, reward_mean=0.475
14: loss=1.554, reward_mean=0.425
15: loss=1.535, reward_mean=0.390
16: loss=1.517, reward_mean=0.420
17: loss=1.498, reward_mean=0.470
18: loss=1.476, reward_mean=0.400
19: loss=1.452, reward_mean=0.440
20: loss=1.436, reward_mean=0.495
21: loss=1.413, reward_mean=0.525
22: loss=1.392, reward_mean=0.470
23: loss=1.373, reward_mean=0.495
24: loss=1.360, reward_mean=0.510
25: loss=1.338, reward_mean=0.575
26: loss=1.325, reward_mean=0.545
27: loss=1.306, reward_mean=0.545
28: loss=1.284, reward_mean=0.580
29: loss=1.265, reward_m

Test part:

In [37]:
print(np.shape(Aeq), np.shape(beq))

(8, 15) (8,)


In [38]:
b_test = np.array(b[:8])

c_test = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

A_test = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

done = 0

counter = 0

MAXCOUNTER=7

x = OBJECT(c_test, A_test, b_test)[0].x

state = STATE(x)

while done != 1 and counter < MAXCOUNTER:


  print("Step: "+str(counter))
  reshaped_x = np.reshape(x, (3, 5))
  #print("Reshaped solution matrix:")
  print(reshaped_x)
  state_t = torch.FloatTensor([state])
  action_scores_t = net(state_t)
  act_probs_t = sm(action_scores_t)
  #print(act_probs_t)
  act_probs = act_probs_t.data.numpy()[0]
  proposed_action=np.argmax(act_probs)
  c_test = OBJECT(c_test, A_test, b_test)[1]
  new = Matrix_Generator(Aeq, b_test, current_state)
  A_test = new[0]
  b_test = new[1]
  #print(c_test, A_test, b_test)
  x = OBJECT(c_test, A_test, b_test)[0].x
  nextstate_, reward, episode_is_done, current_state = next_state(c_test, proposed_action, A_test, b_test)
  #state=next_state.tolist() #changes NumPy array to a Python list
  counter+=1





Step: 0
[[ 1.  0.  1.  1.  0.]
 [ 1.  1. -0.  0.  0.]
 [ 1.  1.  1. -0.  0.]]
Step: 1
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
Step: 2
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
Step: 3
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
Step: 4
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
Step: 5
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
Step: 6
[[ 1. -0.  1.  1. -0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]]
